# Setup

In [163]:
import torch
import argparse
import numpy as np
import utils
import export

In [206]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"

def run_all(args, save=False):
    np.random.seed(args['seed'])
    torch.manual_seed(args['seed'])
    adj, feat, labels, train_mask, val_mask, test_mask = utils.load_data(args['dataset'], args['seed'])
    T_s = T_u = torch.tensor([0])
    if args['T_s'] == -1: T_s = labels
    else:
        T_s = utils.binary(feat[:,args['T_s']].clone())
        feat[:,args['T_s']] = 0
    if args['T_u'] == -1: T_u = labels
    else:
        T_u = utils.binary(feat[:,args['T_u']].clone())
        feat[:,args['T_u']] = 0

    g0_size = int(args['g0_size'] * adj.shape[0])
    g0 = utils.get_g0(g0_size, adj, method=args['g0_method'])
    import attack
    locked_adj = torch.zeros_like(adj)
    print("Attacking with method: " + args['method'])
    if args['method'] == 'CER':
        locked_adj = attack.CER(adj, g0)
    if args['method'] == 'REF':
        budget = (adj.shape[0]) * args['budget_pct']
        locked_adj = attack.REF(adj, g0, budget)
    if args['method'] == 'SLL':
        budget = (adj.shape[0]) * args['budget_pct']
        locked_adj = attack.SLL(adj, feat, g0, T_s, T_u, budget, 100, lr=args['lr'], train_mask=train_mask, val_mask=val_mask, device=device)
    if args['method'] == 'SLL_G':
        budget = (adj.shape[0]) * args['budget_pct']
        sample_size = int(adj.shape[0] * adj.shape[1] * 0.001)
        locked_adj = attack.SLL_G(adj, feat, g0, T_s, T_u, budget, sample_ct=5, sample_size=sample_size, epochs=30, lr=args['lr'], train_mask=train_mask, val_mask=val_mask, device=device)
    sens_g0, sens_gX = utils.evaluate(feat, locked_adj, T_s, g0, train_mask, val_mask, device)
    util_g0, util_gX = utils.evaluate(feat, locked_adj, T_u, g0, train_mask, val_mask, device)
    edges_modified = (locked_adj.cpu() - adj).sum().item()

    adj = adj.to(device)
    sens_clean = utils.evaluate(feat, adj, T_s, g0, train_mask, val_mask, device)
    util_clean = utils.evaluate(feat, adj, T_u, g0, train_mask, val_mask, device)

    results = {
        "sens_g0": sens_g0,
        "sens_gX": sens_gX,
        "util_g0": util_g0,
        "util_gX": util_gX,
        "edges_modified": edges_modified
    }

    results.update(args)
    if save:
        export.saveData(args['location'], results)

In [ ]:
baseargs = {
    'seed': 123,
    # 'method': 'SLL',
    'budget_pct': 0.25,
    'g0_method': 'random',
    # 'g0_size': 0.2,
    'lr': 10,
    'T_s': -1,
    'T_u': 1263,
    'dataset': 'Cora',
    'ptb_rate': 0.5,
    'location': './results3.csv'
 }

baseargs.update({ 'method': 'SLL', 'g0_size': 0.1 })
run_all(baseargs)

baseargs.update({ 'method': 'SLL', 'g0_size': 0.25 })
run_all(baseargs)

baseargs.update({ 'method': 'SLL', 'g0_size': 0.5 })
run_all(baseargs)

baseargs.update({ 'method': 'SLL', 'g0_size': 0.75 })
run_all(baseargs)

In [ ]:
# 'Cora', 'Cora-ML', 'Citeseer', 'Pubmed', 'Polblogs', 'ACM', 'BlogCatalog', 'Flickr', 'UAI'
# TOP 10 cora: [1263, 1301, 1138, 19, 135, 1336, 495, 99, 1123],
# TOP 10 chameleon: [1794, 2753,  528,  598,  415, 2297,  735,   73, 2408, 2539],
# TOP 10 blogcatalog: [6, 10,  9,  7, 17, 18]

In [201]:
baseargs = {
    'seed': 123,
    # 'method': 'SLL',
    'budget_pct': 0.25,
    'g0_method': 'random',
    # 'g0_size': 0.2,
    'lr': 10,
    'T_s': -1,
    'T_u': 1,
    'dataset': 'UAI',
    'ptb_rate': 0.5,
    'location': './results3.csv'
 }

for s in [0.1, 0.25, 0.5, 0.75]:
    baseargs.update({ 'method': 'SLL', 'g0_size': s })
    run_all(baseargs, save=True)
    # break

Loading uai dataset...
G0 with method: random
G0 size: 322
G0 pct: 10.50%
Attacking with method: SLL


100%|██████████| 100/100 [00:08<00:00, 12.11it/s, loss=-4.64e+4]


G0: 20.50%
GX: 33.66%
G0: 54.66%
GX: 60.15%
Loading uai dataset...
G0 with method: random
G0 size: 768
G0 pct: 25.04%
Attacking with method: SLL


100%|██████████| 100/100 [00:08<00:00, 12.23it/s, loss=-5.08e+4]


G0: 29.30%
GX: 34.28%
G0: 56.77%
GX: 59.59%
Loading uai dataset...
G0 with method: random
G0 size: 1501
G0 pct: 48.94%
Attacking with method: SLL


100%|██████████| 100/100 [00:08<00:00, 12.23it/s, loss=-5.18e+4]


G0: 29.45%
GX: 34.93%
G0: 60.03%
GX: 59.71%
Loading uai dataset...
G0 with method: random
G0 size: 2264
G0 pct: 73.82%
Attacking with method: SLL


100%|██████████| 100/100 [00:08<00:00, 12.11it/s, loss=-5.15e+4]


G0: 31.45%
GX: 35.87%
G0: 62.54%
GX: 64.51%


In [202]:
baseargs.update({ 'lr': 20 })
for s in [0.1, 0.25, 0.5, 0.75]:
    baseargs.update({ 'method': 'SLL_G', 'g0_size': s })
    run_all(baseargs, save=True)

Loading uai dataset...
G0 with method: random
G0 size: 322
G0 pct: 10.50%
Attacking with method: SLL_G


100%|██████████| 30/30 [00:02<00:00, 12.88it/s, loss=-1.35e+5]


G0: 29.50%
GX: 36.14%
G0: 49.38%
GX: 52.02%
Loading uai dataset...
G0 with method: random
G0 size: 768
G0 pct: 25.04%
Attacking with method: SLL_G


100%|██████████| 30/30 [00:02<00:00, 12.63it/s, loss=-1.46e+5]


G0: 31.64%
GX: 38.36%
G0: 49.48%
GX: 52.50%
Loading uai dataset...
G0 with method: random
G0 size: 1501
G0 pct: 48.94%
Attacking with method: SLL_G


100%|██████████| 30/30 [00:02<00:00, 13.69it/s, loss=-1.52e+5]


G0: 32.31%
GX: 39.02%
G0: 50.63%
GX: 52.81%
Loading uai dataset...
G0 with method: random
G0 size: 2264
G0 pct: 73.82%
Attacking with method: SLL_G


100%|██████████| 30/30 [00:02<00:00, 12.80it/s, loss=-1.44e+5]


G0: 26.90%
GX: 29.51%
G0: 51.55%
GX: 52.30%


In [207]:
for d in ['Cora', 'BlogCatalog', 'Flickr', 'PolBlogs', 'UAI']:
    for s in [0.1, 0.25, 0.5, 0.75]:
        baseargs.update({ 'method': 'REF', 'g0_size': s, 'dataset': d })
        run_all(baseargs, save=True)

Loading cora dataset...
Selecting 1 largest connected components
G0 with method: random
G0 size: 256
G0 pct: 10.30%
Attacking with method: REF
G0: 66.41%
GX: 82.68%
G0: 98.44%
GX: 97.80%
Loading cora dataset...
Selecting 1 largest connected components
G0 with method: random
G0 size: 625
G0 pct: 25.15%
Attacking with method: REF
G0: 73.60%
GX: 81.83%
G0: 97.76%
GX: 98.01%
Loading cora dataset...
Selecting 1 largest connected components
G0 with method: random
G0 size: 1216
G0 pct: 48.93%
Attacking with method: REF
G0: 76.73%
GX: 79.35%
G0: 98.03%
GX: 98.42%
Loading cora dataset...
Selecting 1 largest connected components
G0 with method: random
G0 size: 1839
G0 pct: 74.00%
Attacking with method: REF
G0: 74.12%
GX: 78.48%
G0: 98.59%
GX: 99.38%
Loading blogcatalog dataset...
G0 with method: random
G0 size: 550
G0 pct: 10.59%
Attacking with method: REF
G0: 55.82%
GX: 57.81%
G0: 100.00%
GX: 100.00%
Loading blogcatalog dataset...
G0 with method: random
G0 size: 1336
G0 pct: 25.71%
Attacking wi

In [209]:
for s in [0.1, 0.25, 0.5, 0.75]:
    baseargs.update({ 'method': 'CER', 'g0_size': s })
    run_all(baseargs, save=True)
for s in [0.1, 0.25, 0.5, 0.75]:
    baseargs.update({ 'method': 'REF', 'g0_size': s })
    run_all(baseargs, save=True)

Loading uai dataset...
G0 with method: random
G0 size: 322
G0 pct: 10.50%
Attacking with method: CER
G0: 6.52%
GX: 41.49%
G0: 49.38%
GX: 50.49%
Loading uai dataset...
G0 with method: random
G0 size: 768
G0 pct: 25.04%
Attacking with method: CER
G0: 3.26%
GX: 43.32%
G0: 49.48%
GX: 55.15%
Loading uai dataset...
G0 with method: random
G0 size: 1501
G0 pct: 48.94%
Attacking with method: CER
G0: 5.46%
GX: 36.21%
G0: 50.63%
GX: 56.58%
Loading uai dataset...
G0 with method: random
G0 size: 2264
G0 pct: 73.82%
Attacking with method: CER
G0: 5.39%
GX: 37.36%
G0: 51.55%
GX: 54.05%
Loading uai dataset...
G0 with method: random
G0 size: 322
G0 pct: 10.50%
Attacking with method: REF
G0: 37.89%
GX: 38.65%
G0: 55.28%
GX: 58.43%
Loading uai dataset...
G0 with method: random
G0 size: 768
G0 pct: 25.04%
Attacking with method: REF
G0: 37.89%
GX: 38.41%
G0: 58.98%
GX: 55.63%
Loading uai dataset...
G0 with method: random
G0 size: 1501
G0 pct: 48.94%
Attacking with method: REF
G0: 35.18%
GX: 37.36%
G0: 50.3